In [9]:
import sys, os
sys.path.append(os.path.abspath(".."))

import pandas as pd
from src.data.dataloader import make_dataset

train_df = pd.read_csv("../data/processed/train.csv")
val_df   = pd.read_csv("../data/processed/val.csv")
test_df  = pd.read_csv("../data/processed/test.csv")


# Build Dataset

In [10]:
from importlib import reload
import src.data.dataloader as dl

reload(dl)


<module 'src.data.dataloader' from '/Users/yunusajib/Desktop/age-detection-ml/src/data/dataloader.py'>

In [12]:
train_df = dl.load_split_csv("../data/processed/train.csv")
train_ds = dl.make_dataset(train_df)


# Inspect one Batch


In [13]:
from src.data.dataloader import make_dataset
import inspect

print(inspect.getsource(make_dataset))


def make_dataset(df, batch_size=32, shuffle=True):
    pixel_strings = df["pixels"].values
    ages = df["age"].values
    ethnicity = df["ethnicity"].values
    genders = df["gender"].values

    dataset = tf.data.Dataset.from_tensor_slices(
        (pixel_strings, ages, ethnicity, genders)
    )

    # Python-side processing
    def _process(pixel_string, age, eth, gender):
        img = load_image_from_pixels(pixel_string.numpy().decode("utf-8"))
        return (
            img,
            tf.cast(age, tf.float32),
            tf.cast(eth, tf.int32),
            tf.cast(gender, tf.int32),
        )

    # TensorFlow wrapper
    def _tf_process(pixel_string, age, eth, gender):
        img, age_out, eth_out, gender_out = tf.py_function(
            func=_process,
            inp=[pixel_string, age, eth, gender],
            Tout=[tf.float32, tf.float32, tf.int32, tf.int32],
        )

        img.set_shape((48, 48, 3))
        age_out.set_shape(())
        eth_out.set_shape(())
    

In [8]:
for images, labels in train_ds.take(1):
    print("Images:", images.shape)
    print("Age:", labels["age"][:5])
    print("Ethnicity:", labels["ethnicity"][:5])
    print("Gender:", labels["gender"][:5])


Images: (32, 48, 48, 3)
Age: tf.Tensor([35. 26. 60. 20. 73.], shape=(5,), dtype=float32)
Ethnicity: tf.Tensor([0 2 3 0 0], shape=(5,), dtype=int32)
Gender: tf.Tensor([0 0 0 1 1], shape=(5,), dtype=int32)


2025-11-29 06:22:54.408184: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
